In [140]:
import numpy as np
from astropy.coordinates import CartesianRepresentation, ITRS, GCRS
from astropy.time import Time
import astropy.units as u
from astropy.utils import iers
import erfa
iers.conf.auto_download = False  # 禁用自动下载

In [ ]:
def keplerian_to_cartesian(a, e, i, raan, argp, nu, mu):
    """
    将开普勒轨道根数转换为笛卡尔位置和速度矢量 (IJK坐标系)。

    参数:
    a : float
        半长轴 (m)
    e : float
        偏心率
    i : float
        轨道倾角 (rad)
    raan : float
        升交点赤经 (rad)
    argp : float
        近地点幅角 (rad)
    nu : float
        真近点角 (rad)
    mu : float
        中心天体重力参数 (m^3/s^2)

    返回:
    r_ijk : numpy array
        位置矢量 [X, Y, Z] (m)
    v_ijk : numpy array
        速度矢量 [Vx, Vy, Vz] (m/s)
    """

    # 1. 计算比角动量 h 和瞬时距离 r
    h = np.sqrt(mu * a * (1 - e**2))
    r = (h**2 / mu) / (1 + e * np.cos(nu))

    # 2. 在PQW轨道平面坐标系中的位置和速度
    r_pqw = np.array([r * np.cos(nu), r * np.sin(nu), 0])
    v_pqw = np.array([-(mu/h) * np.sin(nu), (mu/h) * (e + np.cos(nu)), 0])

    # 3. 构建旋转矩阵 (从PQW到IJK)
    # 注意：代码中使用的是旋转矩阵的转置
    R11 = np.cos(raan)*np.cos(argp) - np.sin(raan)*np.sin(argp)*np.cos(i)
    R12 = -np.cos(raan)*np.sin(argp) - np.sin(raan)*np.cos(argp)*np.cos(i)
    R13 = np.sin(raan)*np.sin(i)
    
    R21 = np.sin(raan)*np.cos(argp) + np.cos(raan)*np.sin(argp)*np.cos(i)
    R22 = -np.sin(raan)*np.sin(argp) + np.cos(raan)*np.cos(argp)*np.cos(i)
    R23 = -np.cos(raan)*np.sin(i)
    
    R31 = np.sin(argp)*np.sin(i)
    R32 = np.cos(argp)*np.sin(i)
    R33 = np.cos(i)
    
    rot_matrix = np.array([[R11, R12, R13],
                           [R21, R22, R23],
                           [R31, R32, R33]])

    # 4. 应用旋转，得到IJK坐标系中的矢量
    r_ijk = rot_matrix @ r_pqw
    v_ijk = rot_matrix @ v_pqw

    return r_ijk, v_ijk

if __name__ == "__main__":
    # 定义地球重力参数 (m^3/s^2)
    mu_earth = 3.986004418e14

    # 输入一组开普勒根数 (示例为一个近地轨道)
    a = 6778000.0  # m (约400km高度)
    e = 0.001
    i = np.deg2rad(51.6)  # 转换为弧度
    raan = np.deg2rad(80.0)
    argp = np.deg2rad(40.0)
    nu = np.deg2rad(30.0)

    # 执行转换
    r_vec, v_vec = keplerian_to_cartesian(a, e, i, raan, argp, nu, mu_earth)

    # 打印结果
    print(f"位置矢量 [X, Y, Z]: {r_vec} 米")
    print(f"速度矢量 [Vx, Vy, Vz]: {v_vec} 米/秒")

位置矢量 [X, Y, Z]: [-3490553.25095484  2967414.22232443  4987205.00104197] 米
速度矢量 [Vx, Vy, Vz]: [-2860.20056896 -6818.00547155  2060.09789316] 米/秒


In [141]:
def eci_to_ecef(x_eci, y_eci, z_eci, utc_time):
    # 计算GAST（Astropy内置方法）
    t = Time(utc_time, scale='utc')
    gast = t.sidereal_time('apparent', 'greenwich').radian
    
    # 构建旋转矩阵
    Rz = np.array([
        [np.cos(gast), np.sin(gast), 0],
        [-np.sin(gast), np.cos(gast), 0],
        [0, 0, 1]
    ])
    
    # 应用旋转
    ecef = Rz @ np.array([x_eci, y_eci, z_eci])
    return ecef

if __name__ == "__main__":
    # 示例：UTC时间2025-09-12 14:52:07
    x_eci, y_eci, z_eci = 1000000, 2000000, 3000000  # 假设ECI坐标
    ecef = eci_to_ecef(x_eci, y_eci, z_eci, '2000-01-01 12:00:00')
    print("ECEF坐标 (m):", ecef)

ECEF坐标 (m): [-1785248.68548046  1346434.97094746  3000000.        ]


In [142]:
def eci_to_ecef_full(x_eci, y_eci, z_eci, utc_time):
    # 定义 GCRS (地心惯性系，近似 ECI)
    gcrs = GCRS(CartesianRepresentation([x_eci, y_eci, z_eci]), obstime=utc_time)

    # 转换到 ITRS (ECEF)
    ecef = gcrs.transform_to(ITRS(obstime=t))
    
    return ecef


if __name__ == "__main__":
    t = Time("2000-01-01 12:00:00", scale="utc")
    x_eci, y_eci, z_eci = 1000000, 2000000, 3000000  # 假设ECI坐标
    ecef = eci_to_ecef_full(x_eci, y_eci, z_eci, '2000-01-01 12:00:00')
    print("ECEF 坐标 (m):", ecef.cartesian)


ECEF 坐标 (m): (-1785232.64476908, 1346634.72812936, 2999919.88443804) 


In [155]:
from astropy.time import Time
from astropy.utils import iers

# 自动下载最新IERS数据
iers_a = iers.IERS_Auto.open()

# 定义时间
t = Time("2000-01-01 12:00:00", scale='utc')

# 获取极移参数
x_p, y_p = iers_a.pm_xy(t)
print(f"x_p = {x_p.to('arcsec')}, y_p = {y_p.to('arcsec')}")

# 小角度近似
x = x_p.to('rad').value
y = y_p.to('rad').value

W = np.array([
    [1, 0, -x],
    [0, 1, -y],
    [x, y, 1]
])

x_p = 0.0433815 arcsec, y_p = 0.3778705 arcsec


In [ ]:
xp = iers_a.pm_x(t).to_value('rad')  # x 极移 (rad)
yp = iers_a.pm_y(t).to_value('rad')  # y 极移 (rad)

xp = x_p.to('rad').value
yp = y_p.to('rad').value

# TT 两部分儒略日
tt1, tt2 = t.tt.jd1, t.tt.jd2
# UT1 两部分儒略日
ut1_1, ut1_2 = t.ut1.jd1, t.ut1.jd2

# 章动+岁差矩阵
pn = erfa.pnm06a(tt1, tt2)

# 地球自转角
era = erfa.era00(ut1_1, ut1_2)
R = np.array([
    [ np.cos(era), np.sin(era), 0.0],
    [-np.sin(era), np.cos(era), 0.0],
    [ 0.0,          0.0,        1.0]
])

# 极移矩阵
# s' 修正
# sp = erfa.sp00(tt1, tt2)
# pom = erfa.pom00(xp, yp, sp)

# 极移矩阵
pom = erfa.pnm00a(t.jd1, t.jd2)

# 合成总矩阵
total = pom @ R @ pn

print("PN 矩阵:\n", pn)
print("R 矩阵:\n", R)
print("POM 矩阵:\n", pom)
print("总矩阵 (ECI→ECEF):\n", total)


PN 矩阵:
 [[ 9.99999998e-01  6.18993850e-05  2.69479054e-05]
 [-6.19001396e-05  9.99999998e-01  2.80031271e-05]
 [-2.69461720e-05 -2.80047951e-05  9.99999999e-01]]
R 矩阵:
 [[ 0.18158511 -0.98337523  0.        ]
 [ 0.98337523  0.18158511  0.        ]
 [ 0.          0.          1.        ]]
POM 矩阵:
 [[ 9.99999998e-01  6.18998304e-05  2.69481136e-05]
 [-6.19005851e-05  9.99999998e-01  2.80030541e-05]
 [-2.69463802e-05 -2.80047221e-05  9.99999999e-01]]
总矩阵 (ECI→ECEF):
 [[ 1.81706852e-01 -9.83352744e-01  4.30582558e-06]
 [ 9.83352743e-01  1.81706852e-01  5.95893094e-05]
 [-5.93797090e-05 -6.59364044e-06  9.99999998e-01]]


In [173]:
vec_gcrs = np.array([1000e3, 2000e3, 3000e3])  # 7000 km
vec_itrs_via_matrix = total @ vec_gcrs
print("\nExample vector (GCRS) -> ITRS via composed matrix:\n", vec_itrs_via_matrix)


Example vector (GCRS) -> ITRS via composed matrix:
 [-1784985.71941402  1346945.21428497  2999927.42765601]


In [106]:
# ECEF 坐标 (m): [-1785232.64476908  1346634.72812936  2999919.88443804] m

In [160]:
from astropy.coordinates import SkyCoord

def get_identity_at_j2000():
    """获取在J2000.0时刻为单位阵的转换"""
    
    # 方法：使用相同的参考系和历元
    j2000 = Time('2000-01-01 12:00:00', scale='utc')
    
    # 创建ICRS坐标（使用J2000.0历元）
    coord_icrs = SkyCoord(
        x=1000*u.m, y=2000*u.m, z=3000*u.m,
        frame='icrs',
        representation_type='cartesian',
        obstime=j2000
    )
    
    # 创建ITRS坐标（也使用J2000.0历元）
    coord_itrs = SkyCoord(
        x=1000*u.m, y=2000*u.m, z=3000*u.m, 
        frame='itrs',
        representation_type='cartesian',
        obstime=j2000
    )
    
    # 这种情况下转换矩阵接近单位阵
    # 因为源和目标使用相同的地球定向参数
    
    return coord_icrs.transform_to('itrs')

# 测试
result = get_identity_at_j2000()
print(f"转换后的坐标: {result.cartesian.xyz.value}")

转换后的坐标: [ 1.35167823e+11  3.05891119e+09 -5.74166675e+10]
